<a href="https://colab.research.google.com/github/clarakl/UoA-GEOG761/blob/main/landslides_patch_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data check


In [1]:
# check_data.py
import pandas as pd
import numpy as np
import os
from datetime import datetime

CSV_PATH = "/content/drive/MyDrive/GEOG761 Machine Learning for Remote Sensing/Group project landslide susceptibility/landslides_with_variables_fixed1.csv"   # CHANGE to your CSV path

def infer_date_col(df):
    # try common names
    for name in ['date','Date','event_date','acq_date','acquisition_date','year']:
        if name in df.columns:
            return name
    # find columns that look like dates
    for col in df.columns:
        if np.issubdtype(df[col].dtype, np.datetime64):
            return col
        if df[col].astype(str).str.match(r'\d{4}-\d{2}-\d{2}').any():
            return col
    return None

def main():
    if not os.path.exists(CSV_PATH):
        raise FileNotFoundError(f"{CSV_PATH} not found. Put your CSV in data/ and update CSV_PATH.")

    df = pd.read_csv(CSV_PATH)
    print("Loaded CSV:", CSV_PATH)
    print("Shape:", df.shape)
    print("Columns:", df.columns.tolist())

    # check lat/lon
    lat_col = None
    lon_col = None
    for c in df.columns:
        if c.lower() in ['latitude','lat','y']:
            lat_col = c
        if c.lower() in ['longitude','lon','lng','x']:
            lon_col = c
    print("Inferred lat column:", lat_col)
    print("Inferred lon column:", lon_col)
    if lat_col is None or lon_col is None:
        print("WARNING: Lat/Lon columns not clearly found. Please provide column names for latitude and longitude.")
    else:
        print("Lat/Lon sample (first 5):")
        print(df[[lat_col, lon_col]].head())

    # infer date
    date_col = infer_date_col(df)
    print("Inferred date column:", date_col)
    if date_col:
        try:
            df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
            print("Date conversion done. Missing dates:", df[date_col].isna().sum())
            # print yearly distribution
            df['year'] = df[date_col].dt.year
            print("Year counts (top):")
            print(df['year'].value_counts().sort_index())
        except Exception as e:
            print("Could not parse dates:", e)
    else:
        print("No date column inferred. You'll need to provide an acquisition date column or supply imagery filtered by date externally.")

    # check for label column
    potential_labels = [c for c in df.columns if c.lower() in ['landslide','valid landslide','label','target','y','class']]
    print("Potential label columns:", potential_labels)
    if potential_labels:
        label = potential_labels[0]
        print("Label distribution:")
        print(df[label].value_counts(dropna=False))

    # quick spatial bounds
    if lat_col and lon_col:
        minlat, maxlat = df[lat_col].min(), df[lat_col].max()
        minlon, maxlon = df[lon_col].min(), df[lon_col].max()
        print(f"Spatial extent: lat [{minlat:.4f}, {maxlat:.4f}], lon [{minlon:.4f}, {maxlon:.4f}]")

    print("\nIf you have imagery, place it under data/imagery/ as GeoTIFFs named with a date or keep a metadata file mapping tiles to dates.")
    print("Next (Step 2) I will show how to build image patches around each lat/lon using rasterio, and how to filter imagery by date so we only use imagery prior to the landslide event.")

if __name__ == "__main__":
    main()


Loaded CSV: /content/drive/MyDrive/GEOG761 Machine Learning for Remote Sensing/Group project landslide susceptibility/landslides_with_variables_fixed1.csv
Shape: (23852, 16)
Columns: ['ID', 'Area Maximum', 'Valid Landslide', 'Latitude', 'Longitude', 'CURVATURE', 'TWI', 'SLOPE_deg', 'DEM', 'SLOPE_deg.1', 'ASPECT_deg', 'LANDCOVER_CODE', 'ASPECT_rad', 'ASPECT_sin', 'ASPECT_cos', 'SLOPE_rad']
Inferred lat column: Latitude
Inferred lon column: Longitude
Lat/Lon sample (first 5):
    Latitude   Longitude
0 -36.819859  174.746298
1 -36.798134  174.684934
2 -36.932751  174.744548
3 -36.852885  174.792092
4 -36.809204  174.718141
Inferred date column: None
No date column inferred. You'll need to provide an acquisition date column or supply imagery filtered by date externally.
Potential label columns: ['Valid Landslide']
Label distribution:
Valid Landslide
1    11926
0    11926
Name: count, dtype: int64
Spatial extent: lat [-37.2516, -36.1604], lon [174.2321, 175.2778]

If you have imagery, plac

# Extract patches

How it works:

- Sentinel-2 SR provides L2A reflectance at 10 m.

- The .median() composite merges cloud-free pixels from 2019–2022.

- Each point.buffer(464 m) yields a ~928 m window around the coordinate.

- Patches are exported to Google Drive → “GEE_Landslide_Patches” folder.

In [2]:
# Set up GEE API
import ee
ee.Authenticate()
ee.Initialize(project='clara-geog761-tryout-1')

Now, we will transform the numerical long-lat  into GEE points that are georeferenced. The cell after this visualizes these points.

In [3]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/GEOG761 Machine Learning for Remote Sensing/Group project landslide susceptibility/landslides_with_variables_fixed1.csv")

In [6]:
# Convert to ee.FeatureCollection
def row_to_feature(row):
    geom = ee.Geometry.Point(float(row['Longitude']), float(row['Latitude']))
    # Make sure 'Valid Landslide' is integer and has no nulls for filtering
    label = row['Valid Landslide']
    if pd.notna(label):
        return ee.Feature(geom, {'id': int(row.name), 'label': int(label)})
    else:
      print("AAAAAAAAAAAAAA")
    return None

features = [row_to_feature(r) for _, r in df.iterrows()]
features = [f for f in features if f is not None] # Remove null features
fc = ee.FeatureCollection(features)

print("Feature collection created with", fc.size().getInfo(), "points.")

Feature collection created with 23852 points.


In [5]:
import geemap

# Create an interactive map
Map = geemap.Map()

# Define visualization parameters (e.g., color the points red)
vis_params = {'color': 'red'}

# Add the FeatureCollection to the map
Map.addLayer(fc, vis_params, 'Landslide Points')

# Center the map view on your points with a zoom level of 8
Map.centerObject(fc, 8)

# Display the map
Map

Map(center=[-36.68817951291026, 174.66389329965313], controls=(WidgetControl(options=['position', 'transparent…

In [7]:
# Sentinel-2 Level-2A (Surface Reflectance)
collection = (ee.ImageCollection('COPERNICUS/S2_SR')
              .filterDate('2022-01-01', '2022-12-31')
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)))

median_image = collection.median().select(['B2','B3','B4','B8','B11','B12'])  # Blue, Green, Red, NIR

Calculate area needed by taking the mean maximum area from all of the landslides

In [8]:
import numpy as np

# Define patch size
df['Area Maximum'] = pd.to_numeric(df['Area Maximum'], errors='coerce')
patch_size_m = np.sqrt(df['Area Maximum'].mean(skipna=True) * 10_000)
half_width = patch_size_m / 2
print(f"Suggested patch width: {patch_size_m:.0f} meters")

Suggested patch width: 928 meters


In [9]:
# Create a smaller collection for the proof-of-concept export.
# Define the number of samples to take from each class
sample_size = 1250

# Filter the collection for each class
positives = fc.filter(ee.Filter.eq('label', 1)).limit(sample_size)
negatives = fc.filter(ee.Filter.eq('label', 0)).limit(sample_size)

# Merge the two limited collections into one
fc_limited = positives.merge(negatives)

# Shuffle the collection to mix the positive and negative samples
fc_limited = fc_limited.randomColumn()
fc_limited = fc_limited.sort('random')
print(f"Limited feature collection to {fc_limited.size().getInfo()} points.")

def create_and_tag_patch(feature):
    """Extracts a patch and sets its ID and label as properties."""
    patch = median_image.clip(feature.geometry().buffer(half_width).bounds())
    # Set() attaches metadata to the image patch for later use
    return patch.set({
        'id': feature.get('id'),
        'label': feature.get('label')
    })

# Map the new function over the LIMITED collection
tagged_image_patches = fc_limited.map(create_and_tag_patch)

print(f"Created a collection of {tagged_image_patches.size().getInfo()} tagged patches.")

Limited feature collection to 2500 points.
Created a collection of 2500 tagged patches.


In [11]:
# Making sure that the patches have their labels

print("Labels of first 5 patches:", tagged_image_patches.limit(5).aggregate_array('label').getInfo())
print("Nr of negative and positive samples:", tagged_image_patches.aggregate_histogram('label').getInfo())

[1, 0, 1, 1, 1]
{'0': 1250, '1': 1250}


Now we will create these patches in GEE.


In [12]:
# Convert the server-side collection to a client-side list to loop through it.
# This is safe for 2,500 items.
patch_list = tagged_image_patches.toList(tagged_image_patches.size())

# Get the number of patches to process
num_patches = patch_list.size().getInfo()

In [14]:
# Checking what the entries in the list look like to make sure we're on the right track

first_patch_info = ee.Image(patch_list.get(0)).getInfo()
print(first_patch_info)

{'type': 'Image', 'bands': [{'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [1, 1], 'origin': [174, -37], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [1, 1], 'origin': [174, -37], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [1, 1], 'origin': [174, -37], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B8', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [1, 1], 'origin': [174, -37], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B11', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [1, 1], 'origin': [174, -37], 'crs': 'EPSG:4326', 'crs_transform': [1

In [ ]:
# Loop through the list and create an export task for each patch
for i in range(num_patches):
    # Get the image patch from the list
    image = ee.Image(patch_list.get(i))

    # Get its metadata
    patch_id = image.get('id').getInfo()
    patch_label = image.get('label').getInfo()

    # --- This is the key step ---
    # Create a descriptive filename that includes the ID and the label.
    filename = f"patch_id_{patch_id}_label_{patch_label}"

    # Define the export task for this single patch
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'Export_Patch_{patch_id}',  # Each task needs a unique description
        folder='GEE_Landslide_Patches',
        fileNamePrefix=filename,
        scale=10,
        fileFormat='GeoTIFF' # GeoTIFF is a standard image format
    )
    if i%100 == 0:
      print(f"Exporting patch {i+1}/{num_patches}...")

    # Start the task
    task.start()

print(f"All {num_patches} tasks have been submitted.")
print("Monitor their progress in the 'Tasks' tab of the GEE Code Editor.")

Exporting patch 1/2500...


In [ ]:
# Visualizing one patch

import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np

filepath = '/content/drive/MyDrive/GEE_Landslide_Patches/patch_id_1171_label_1.tif'

with rasterio.open(filepath) as src:
    # Read the red, green, and blue bands into a 3D array
    # Note: Sentinel-2 band numbers might be different, e.g., B4, B3, B2 are often bands 4, 3, 2.
    # We'll assume the first three bands are the ones we want for simplicity here.
    # You may need to adjust the numbers in read([1, 2, 3])
    rgb = src.read([1, 2, 3])

    # Function to normalize bands for display
    def normalize(array):
        array_min, array_max = array.min(), array.max()
        return ((array - array_min) / (array_max - array_min))

    # Normalize each band to the 0-1 range for proper RGB display
    red_normalized = normalize(rgb[0])
    green_normalized = normalize(rgb[1])
    blue_normalized = normalize(rgb[2])

    # Stack the bands back together
    rgb_normalized = np.dstack((red_normalized, green_normalized, blue_normalized))

    # Display the true-color image
    plt.imshow(rgb_normalized)
    plt.show()

--------
#STOP HERE,  CODE AFTER THIS IS WORK IN PROGRESS FOR THE MODEL ITSELF
----

This code prepares a machine learning dataset by mapping over a collection of geographic points (fc). For each point, it extracts a corresponding square patch from a satellite image, then adds the point's label (e.g., landslide or not) directly onto the patch as a new data band. To satisfy the GEE exporter's requirement for a single input image, all these individual, labeled patches are stitched together into a temporary virtual image using .mosaic(). Finally, the Export.image function is configured with patchDimensions to cut this virtual image back up into the original patches, saving them in the highly efficient TFRecord format, which is ideal for training deep learning models.

In [ ]:
import ee

# --- Assumed Setup ---
# This code assumes you have authenticated with Earth Engine and have the
# following variables defined from the previous steps:
# ee.Initialize()
# fc = your ee.FeatureCollection with the 'label' property for each point
# collection = your initial ee.ImageCollection('COPERNICUS/S2_SR')
# patch_size_m = the width of your patches in meters (e.g., 316)
# --- 1. Define Helper Function and Final Image ---
# --- 1. Define Helper Function and Final Image ---

# This function was defined in a previous step
def extract_rectangular_patch(image, feature, half_width_m=100):
    """Extracts a rectangular patch from an image."""
    geom = feature.geometry().buffer(half_width_m).bounds()
    patch = image.clip(geom)
    return patch

# Create the median composite image with all the necessary spectral bands
bands_to_select = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
median_image = collection.median().select(bands_to_select)
half_width = patch_size_m / 2

# --- 2. Create Labeled Image Patches ---

# Define a function that extracts a patch AND adds the label as a new band.
def extract_and_label_patch(feature):
    patch = extract_rectangular_patch(median_image, feature, half_width)
    # Create a constant image from the 'label' property of the feature.
    label_image = ee.Image.constant(feature.get('label')).rename('label')
    # Add this new 'label' band to the image patch.
    return patch.addBands(label_image)

# Filter the collection to only include features that have a non-null 'label' property.
filtered_fc = fc.filter(ee.Filter.notNull(['label']))

# Now, map the function over the CLEANED collection
labeled_patches = ee.ImageCollection(filtered_fc.map(extract_and_label_patch))

# This is the key step: convert the collection of patches into a single image.
image_to_export = labeled_patches.mosaic()

# The rest of your code (mosaic, export, etc.) remains the same, but you may
# want to update the export_region to use the filtered collection.
export_region = filtered_fc.geometry().bounds()

# Calculate the export dimensions based on the patch size and the
# satellite's native resolution (10m for Sentinel-2).
native_pixel_dimensions = int(patch_size_m / 10)

# Define the export task configuration
task = ee.batch.Export.image.toDrive(
  image=image_to_export,
  description='Landslide_TFRecord_Export_Fixed',
  folder='GEE_Landslide_Exports',
  fileNamePrefix='landslide_data',
  region=export_region,
  scale=10,
  maxPixels=1e13,
  fileFormat='TFRecord',
  formatOptions={
    'patchDimensions': [native_pixel_dimensions, native_pixel_dimensions],
    'compressed': True
  }
)

# Start the export task
task.start()

print("Export task started successfully!")
print("Monitor the progress in the 'Tasks' tab of the Google Earth Engine Code Editor.")


Export task started successfully!
Monitor the progress in the 'Tasks' tab of the Google Earth Engine Code Editor.


# Dataset preparation

## Step 1: Access data in google drive

In [ ]:
import tensorflow as tf
import glob

# Get a list of all your TFRecord files. The '*' handles cases where GEE
# creates multiple files (e.g., landslide_data-00000.tfrecord, etc.)
folder_path = '/content/drive/MyDrive/GEE_Landslide_Exports/'
tfrecord_files = glob.glob(f"{folder_path}*.tfrecord")

print(f"Found {len(tfrecord_files)} TFRecord files.")

## Step 2: Decode the binary data for each band in the TFRecord file

Once the big blueprint image is loaded into the drive from GEE, we need to seperate the features from the label. The output type of this operation are **3d tensors** (width, length, bands). The features are the images, the target is the label (hwich is pased as a band).


In [ ]:
# --- You must define these variables based on your export ---
# This is the 'native_pixel_dimensions' variable from your GEE script
PATCH_SIZE = 114 # Example: int(patch_size_m / 10)
# All the bands you exported, INCLUDING the 'label' band
BANDS = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'label']

def parse_and_split_tfrecord(serialized_example):
    """Parses a single TFRecord, stacks bands, and splits into features and label."""

    # 1. Define the feature description (the "recipe" for parsing)
    # Each band is a 2D image patch of a fixed size.
    feature_description = {
        band: tf.io.FixedLenFeature([PATCH_SIZE, PATCH_SIZE], tf.float32) for band in BANDS
    }

    # 2. Parse the input `serialized_example` using the dictionary
    parsed_features = tf.io.parse_single_example(serialized_example, feature_description)

    # 3. Stack the individual bands into a single 3D tensor
    # The order of bands here is critical.
    stacked_tensor = tf.stack([parsed_features[b] for b in BANDS], axis=-1)

    # 4. Split the tensor into features (image) and label
    # Features are all bands except the last one
    features = stacked_tensor[:, :, :-1]  # Shape: [PATCH_SIZE, PATCH_SIZE, 6]

    # The label is a single value from the last band
    label = stacked_tensor[0, 0, -1]

    # Ensure the label is an integer (standard for classification)
    label = tf.cast(label, tf.int32)

    return features, label

In [ ]:
# preview_patches_gee.py
import ee, geemap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime


# ------------------------------
# CONFIG
# ------------------------------
CSV_PATH = "landslides_with_variables2.csv"
NUM_PREVIEW = 3
PATCH_WIDTH_M = 928
HALF_PATCH = PATCH_WIDTH_M / 2
SCALE = 10
MONTHS_WINDOW = 3           # shorter for faster testing
END_DATE_GLOBAL = "2022-12-31"
CLOUD_PROB_THRESH = 40
SCL_MASK_VALUES = [3, 8, 9, 10]

# ------------------------------
# FIXED CLOUD-JOIN FUNCTION
# ------------------------------
def add_cloudprob_to_s2collection(s2_sr):
    cloud_coll = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
    time_filter = ee.Filter.maxDifference(
        difference=24 * 60 * 60 * 1000,
        leftField='system:time_start',
        rightField='system:time_start'
    )
    joined = ee.Join.saveBest('cloud', 'timeDiff').apply(
        primary=s2_sr,
        secondary=cloud_coll,
        condition=time_filter
    )
    def merge_cloud(img):
        cloud = ee.Image(img.get('cloud'))
        return ee.Image(img).addBands(cloud.select('probability'))
    return ee.ImageCollection(joined.map(merge_cloud))

def mask_clouds(img):
    scl = img.select('SCL')
    mask_scl = scl.remap(SCL_MASK_VALUES, [0]*len(SCL_MASK_VALUES), 1).eq(1)
    img = img.updateMask(mask_scl)
    prob = img.select('probability')
    mask_cp = prob.lte(CLOUD_PROB_THRESH)
    return img.updateMask(mask_cp)

# ------------------------------
# COMPOSITE BUILDER WITH PROGRESS PRINTS
# ------------------------------
def make_composite(lon, lat, end_date=END_DATE_GLOBAL, months=MONTHS_WINDOW):
    end = ee.Date(end_date)
    start = end.advance(-months, 'month')
    geom = ee.Geometry.Point([lon, lat]).buffer(HALF_PATCH)
    print(f"  Date range: {start.format('YYYY-MM-dd').getInfo()} → {end.format('YYYY-MM-dd').getInfo()}")

    s2 = (ee.ImageCollection('COPERNICUS/S2_SR')
          .filterDate(start, end)
          .filterBounds(geom)
          .select(['B2','B3','B4','B8','SCL']))

    # Count how many raw images exist before cloud join
    count_raw = s2.size().getInfo()
    print(f"  Raw Sentinel-2 SR images found: {count_raw}")

    if count_raw == 0:
        return None

    s2 = add_cloudprob_to_s2collection(s2)
    s2 = s2.map(mask_clouds)

    # Count after masking
    count_masked = s2.size().getInfo()
    print(f"  Images remaining after masking: {count_masked}")

    # Show a few timestamps
    img_list = s2.toList(5)
    timestamps = [ee.Image(img_list.get(i)).date().format('YYYY-MM-dd').getInfo()
                  for i in range(min(5, count_masked))]
    print("  Example image dates:", timestamps)

    if count_masked == 0:
        return None

    comp = s2.median().divide(10000).select(['B2','B3','B4','B8'])
    return comp.clip(geom)

# ------------------------------
# EXTRACT + PREVIEW
# ------------------------------
def extract_numpy_patch(image, lon, lat):
    geom = ee.Geometry.Point([lon, lat]).buffer(HALF_PATCH)
    d = image.sampleRectangle(region=geom, defaultValue=0).getInfo()
    bands = ['B2','B3','B4','B8']
    arrs = [np.array(d[b]) for b in bands if b in d]
    if len(arrs) != 4:
        return None
    return np.stack(arrs, axis=-1)

df = pd.read_csv(CSV_PATH)

for i, row in df.head(NUM_PREVIEW).iterrows():
    lon, lat = row['Longitude'], row['Latitude']
    print(f"\n[{i}] Building composite for ({lat:.4f}, {lon:.4f}) …")
    comp = make_composite(lon, lat)
    if comp is None:
        print("  ⚠️  No valid images, skipping.")
        continue
    patch = extract_numpy_patch(comp, lon, lat)
    if patch is None or np.all(patch == 0):
        print("  ⚠️  Patch empty or invalid.")
        continue

    rgb = patch[:, :, [2,1,0]]
    vmin, vmax = np.percentile(rgb[rgb>0], (2, 98))
    rgb = np.clip((rgb - vmin) / (vmax - vmin), 0, 1)
    plt.figure(figsize=(6,6))
    plt.imshow(rgb)
    plt.title(f"Sample {i} ({lat:.3f}, {lon:.3f})")
    plt.axis('off')
    plt.show()


[0] Building composite for (-36.8199, 174.7463) …
  Date range: 2022-09-30 → 2022-12-31
  Raw Sentinel-2 SR images found: 19
  Images remaining after masking: 19


KeyboardInterrupt: 

In [ ]:

# --- Parameters ---
PATCH_WIDTH_M = 928   # from your mean area estimate
HALF_PATCH = PATCH_WIDTH_M / 2
POINT = ee.Geometry.Point(174.7463, -36.8199)

# --- Date range ---
start = '2022-09-30'
end   = '2022-12-31'

# --- Sentinel-2 Surface Reflectance ---
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(POINT)
    .filterDate(start, end)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
)

# Cloud/shadow masking function
def mask_s2_sr(image):
    cloud_prob = image.select('SCL').eq(3).Or(image.select('SCL').eq(8))
    snow = image.select('SCL').eq(11)
    return image.updateMask(cloud_prob.Not()).updateMask(snow.Not())

masked = collection.map(mask_s2_sr)

# Median composite
median_img = masked.median().select(['B4', 'B3', 'B2', 'B8'])

# Clip to a 928m square patch
patch_geom = POINT.buffer(HALF_PATCH).bounds()
patch_img = median_img.clip(patch_geom)

# --- Generate a download URL ---
url = patch_img.getDownloadURL({
    'scale': 10,           # Sentinel-2 native resolution
    'crs': 'EPSG:4326',
    'region': patch_geom
})

print("✅ Download URL for the patch:\n", url)

✅ Download URL for the patch:
 https://earthengine.googleapis.com/v1/projects/clara-geog761-tryout-1/thumbnails/d30f48b71ad7099d95e9160340494ed3-fa39a4be6f1e77d68e9a34627544dcbf:getPixels


In [ ]:
Map = geemap.Map()
Map.centerObject(fc, 10)
Map.addLayer(median_image, {'bands':['B4','B3','B2'], 'min':0, 'max':3000}, 'Sentinel-2 pre-2023')
Map.addLayer(fc, {}, 'Landslide points')
Map


Map(center=[-36.73285870698243, 174.68500559550705], controls=(WidgetControl(options=['position', 'transparent…